In [219]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import requests
import json
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.cluster import KMeans
import requests
from pylab import rcParams
rcParams['figure.figsize'] = 9, 8

In [220]:
data = requests.get('http://localhost:8000/api/tingkatbutuhguru').json() 

In [221]:
sekolah = []
siswa = []
guru = []
wilayah = []
all_data = [] 
cluster = data['results']

In [222]:
for val in cluster:
    sekolah.append(int(val['sekolah']))
    siswa.append(int(val['siswa']))
    guru.append(int(val['guru']))
    wilayah.append(val['wilayah'])
    all_data.append({"sekolah":int(val['sekolah']),"siswa":int(val['siswa']),"guru":int(val['guru'])})

In [223]:
df = pd.DataFrame(all_data)
df_wilayah = pd.DataFrame(wilayah)

In [224]:
df_wilayah

,0
0,Kab Bogor
1,Kab Sukabumi
2,Kab Cianjur
3,Kab Bandung
4,Kab Sumedang
5,Kab Garut
6,Kab Tasikmalaya
7,Kab Ciamis
8,Kab Kuningan
9,Kab Majalengka


In [225]:
df.describe()

,guru,sekolah,siswa
count,27.000000,27.000000,27.000000
mean,588.777778,168.555556,60995.555556
std,301.385051,114.360877,40483.397888
min,35.000000,22.000000,10912.000000
25%,357.500000,79.500000,34353.500000
50%,565.000000,155.000000,56827.000000
75%,779.000000,243.000000,79898.500000
max,1223.000000,538.000000,189915.000000


In [226]:
clustervar=df.copy()
clustervar['guru']=preprocessing.scale(clustervar['guru'].astype('float64'))

In [227]:
clustervar['siswa']=preprocessing.scale(clustervar['siswa'].astype('float64'))

In [228]:
clustervar['sekolah']=preprocessing.scale(clustervar['sekolah'].astype('float64'))

In [229]:
clus_train = clustervar

In [230]:
clustervar


,guru,sekolah,siswa
0,1.745461,3.292053,3.245164
1,0.859581,1.180189,1.150499
2,2.144445,0.734648,1.246052
3,0.467360,0.957418,0.781351
4,1.383671,0.592075,1.597932
5,0.791957,1.206921,-1.260705
6,0.203625,0.306928,0.636486
7,1.569638,0.262374,0.777047
8,0.937349,-0.120791,0.315165
9,0.494410,0.850488,0.284531


In [231]:
from scipy.spatial.distance import cdist

In [232]:
clusters=range(1,11)
meandist=[]

In [233]:
for k in clusters:
    model=KMeans(n_clusters=k)
    model.fit(clus_train)
    clusassign=model.predict(clus_train)
    meandist.append(sum(np.min(cdist(clus_train, model.cluster_centers_, 'euclidean'), axis=1))
    / clus_train.shape[0])

In [234]:
plt.plot(clusters, meandist)
plt.xlabel('Number of clusters')
plt.ylabel('Average distance')
plt.title('Selecting k with the Elbow Method')

Text(0.5,1,'Selecting k with the Elbow Method')

In [235]:
model3=KMeans(n_clusters=3)
model3.fit(clus_train)
clusassign=model3.predict(clus_train)

In [236]:
plt.xlabel('Canonical variable 1')
plt.ylabel('Canonical variable 2')
plt.title('Scatterplot of Canonical Variables for 2 Clusters')

Text(0.5,1,'Scatterplot of Canonical Variables for 2 Clusters')

In [237]:
clus_train.reset_index(level=0, inplace=True)

cluslist=list(clus_train['index'])

labels=list(model3.labels_)

newlist=dict(zip(cluslist, labels))
newlist
newclus= pd.DataFrame.from_dict(newlist, orient='index')
newclus

newclus.columns = ['cluster']
clus_train


,index,guru,sekolah,siswa
0,0,1.745461,3.292053,3.245164
1,1,0.859581,1.180189,1.150499
2,2,2.144445,0.734648,1.246052
3,3,0.467360,0.957418,0.781351
4,4,1.383671,0.592075,1.597932
5,5,0.791957,1.206921,-1.260705
6,6,0.203625,0.306928,0.636486
7,7,1.569638,0.262374,0.777047
8,8,0.937349,-0.120791,0.315165
9,9,0.494410,0.850488,0.284531


In [238]:
newclus.reset_index(level=0, inplace=True)

merged_train=pd.merge(clus_train, newclus, on='index')
merged_train.head(n=100)

merged_train.cluster.value_counts()

0    15
1    11
2     1
Name: cluster, dtype: int64

In [239]:
newclus

,index,cluster
0,0,2
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1


In [240]:
result1 = pd.concat([df,df_wilayah,newclus], axis=1)

In [241]:
result = result1.rename(columns = {0 : 'kabupaten'})

In [243]:
result

,guru,sekolah,siswa,kabupaten,index,cluster
0,1105,538,189915,Kab Bogor,0,2
1,843,301,106701,Kab Sukabumi,1,1
2,1223,251,110497,Kab Cianjur,2,1
3,727,276,92036,Kab Bandung,3,1
4,998,235,124476,Kab Sumedang,4,1
5,823,304,10912,Kab Garut,5,1
6,649,203,86281,Kab Tasikmalaya,6,1
7,1053,198,91865,Kab Ciamis,7,1
8,866,155,73516,Kab Kuningan,8,1
9,735,264,72299,Kab Majalengka,9,1
